In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Asthma"
cohort = "GSE270312"

# Input paths
in_trait_dir = "../../input/GEO/Asthma"
in_cohort_dir = "../../input/GEO/Asthma/GSE270312"

# Output paths
out_data_file = "../../output/preprocess/Asthma/GSE270312.csv"
out_gene_data_file = "../../output/preprocess/Asthma/gene_data/GSE270312.csv"
out_clinical_data_file = "../../output/preprocess/Asthma/clinical_data/GSE270312.csv"
json_path = "../../output/preprocess/Asthma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The rs6967330 minor allele in CDHR3 increased CRS exacerbations and is associated with an exaggerated interferon response to RV-A and RV-C infections"
!Series_summary	"Background: Adults with at least one copy of the minor allele in the rs6967330 SNP (AA/AG) in the rhinovirus (RV) receptor Cadherin related family member 3 gene (CDHR3), have a higher risk for CRS than those with two copies of the major allele (GG)."
!Series_summary	"Objective: To determine if the rs6967330 SNP increased the risk for acute exacerbations of chronic rhinosinusitis (AECRS) in adults and identify if their nasal cells showed a distinct pathophysiologic process activated by RV infection."
!Series_summary	"Methods: We recruited adults with CRS (AG/AA,n=17; GG,n=37) and at baseline collected sinonasal outcome tests (SNOT-22), objective endoscopy scores, and nasal brushings for cells and RV viral detection. Subjects were contacted every two weeks for AECRS over one year, and

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background, the study involves RNA transcriptome analysis,
# so this dataset should contain gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For the trait (Asthma), we find it in row 3
trait_row = 3

# For age, there's no information in the sample characteristics
age_row = None

# For gender, we find it in row 2
gender_row = 2

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert asthma status to binary: 1 for Yes, 0 for No."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if isinstance(value, str):
        if value.lower() == 'yes':
            return 1
        elif value.lower() == 'no':
            return 0
    
    return None

def convert_age(value):
    """Convert age to continuous value."""
    # This function is defined but not used since age data is not available
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary: 0 for Female, 1 for Male."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if isinstance(value, str):
        if value.lower() == 'female':
            return 0
        elif value.lower() == 'male':
            return 1
    
    return None

# 3. Save Metadata for initial filtering
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only if trait_row is not None
if trait_row is not None:
    # Assuming clinical_data is the input DataFrame
    # We need to read it first from the cohort directory
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age if age_row is not None else None,
            gender_row=gender_row,
            convert_gender=convert_gender if gender_row is not None else None
        )
        
        # Preview the DataFrame
        print("Preview of selected clinical features:")
        print(preview_df(selected_clinical_df))
        
        # Save to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Asthma/GSE270312/GSE270312_series_matrix.txt.gz
Gene data shape: (832, 90)
First 20 gene/probe identifiers:
Index(['ABCF1', 'ACE', 'ACKR2', 'ACKR3', 'ACKR4', 'ACOX1', 'ACSL1', 'ACSL3',
       'ACSL4', 'ACVR1', 'ADAM17', 'ADAR', 'ADGRE5', 'ADGRG3', 'ADORA2A',
       'AGT', 'AHR', 'AIF1', 'AIM2', 'AKT1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the gene expression data
# These identifiers (ABCF1, ACE, ACKR2, etc.) are standard human gene symbols
# They match official HGNC gene symbols and do not need mapping

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# First, re-extract the necessary files from the cohort directory
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Read background information and clinical data again to ensure we have the correct data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Get the gene data again
gene_data = get_genetic_data(matrix_file)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Using the correct trait_row and gender_row identified in step 2
# Extract clinical features using the appropriate conversion functions
selected_clinical_data = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=3,  # Using trait_row = 3 for asthma status
    convert_trait=lambda value: 1 if isinstance(value, str) and 'yes' in value.lower().split(':')[-1].strip() else 0,
    age_row=None,  # No age data available
    convert_age=None,
    gender_row=2,  # Using gender_row = 2
    convert_gender=lambda value: 1 if isinstance(value, str) and 'male' in value.lower().split(':')[-1].strip() else 0
)

# Save the processed clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_data.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_data, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from sinonasal epithelial cells with Asthma status information."
)

# Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Normalized gene data saved to ../../output/preprocess/Asthma/gene_data/GSE270312.csv
Clinical data saved to ../../output/preprocess/Asthma/clinical_data/GSE270312.csv
Linked data shape: (90, 834)
Linked data preview (first 5 rows, 5 columns):
            Asthma  Gender    ABCF1      ACE    ACKR2
GSM8339381     0.0     1.0  9.57557  5.71144  5.79512
GSM8339382     0.0     1.0  9.05703  5.50984  5.42685
GSM8339383     0.0     1.0  9.07081  5.17455  5.68615
GSM8339384     0.0     1.0  9.56418  5.68938  5.53547
GSM8339385     0.0     1.0  9.19873  5.16993  4.88070
Data shape after handling missing values: (90, 834)


For the feature 'Asthma', the least common label is '0.0' with 39 occurrences. This represents 43.33% of the dataset.
The distribution of the feature 'Asthma' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 90 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Linked data saved to ../../output/preprocess/Asthma/GSE270312.csv
